In [3]:
import numpy as np 
import pandas as pd 
import requests
import geocoder # import geocoder
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Save the table from wikipedia on a pd.dataframe called 'df'

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]

## Drop all rows of 'df' where 'Boroough' is 'Not assigned'

In [5]:
df_drop=df.drop(df[df['Borough']=='Not assigned'].index)

## Check if there are rows with same 'Postal Code'

In [7]:
print(df_drop.shape)
len(df_drop['Postal Code'].unique())


(103, 3)


103

_since the number of rows is the same with the number of unique values in the column 'Postal Code', it can be assumed that there are no duplicates of the same postal code_

## Check if there are any 'Not assigned' values in 'Neighbourhoods' column

In [8]:
isin=df_drop.Neighbourhood.isin(df_drop.Neighbourhood=='Not assigned')
isin.value_counts()

False    103
Name: Neighbourhood, dtype: int64

_since that all rows return a 'False' it can be assumed that there are no 'neighbourhoods' witht the value 'Not assigned'_

## Reset the index of the dataframe and print its shape 

In [9]:
df_drop.reset_index(drop=True, inplace=True)

In [10]:
df_drop.shape

(103, 3)

In [11]:
df_drop.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Since geocoder Google was not working at all for me, I opted for Arcgis to obtain the Postal Codes. This provider is working flowlessly so no need for the WHILE loop 

In [12]:
coords=[]
for cd in df_drop['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(cd))
    lat_lng_coords = g.latlng
    coords.append(lat_lng_coords)


## Appending the Longitude and Latitude to the Dataframe (df_drop)

In [15]:
latitude=[]
longitude=[]
for i in coords:
    latitude.append(i[0])
    longitude.append(i[1])
df_drop['Latitude']=pd.Series(latitude)
df_drop['Longitude']=pd.Series(longitude)

In [16]:
display(df_drop.head())


,Postal Code,Borough,Neighbourhood,Latitude,longitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,-79.39188
